## **Prediction of house prices using multivariate linear regression**
## *By utitilizing only Pandas and NumPy libraries*

# **1. Introduction**

This is a classic example for beginners in machine learning. The dataset contains the **house sales for King County, USA, between May 2014 and May 2015**. You can find the full dataset on [Kaggle](https://www.kaggle.com/harlfoxem/housesalesprediction). Before continuing, it is highly recommended that you click the aforementioned link and get to know the original dataset.

Our aim is to apply multivariate linear regression so as to predict the price of any given house in that area, given a number of its features (e.g. number of bathrooms, year on which the house was built, etc.). For this purpose however, **only NumPy and Pandas libraries are used** (as well as Matplotlib and Seaborn for the visualizations). Although some may find the code to be unnecessarily lengthy and detailed, I believe that it can provide the reader with a good grasp of what is really happening in such models and thus, can have great **educational value** (especially for beginners). The code is largely based on Stanford's Machine Learning course on [Coursera](https://www.coursera.org/learn/machine-learning/) by Andrew Ng and can be very helpful for those who have taken it and wish to see an implementation in Python.

If you have any questions or feedback, please feel free to contact me. Please do not forget to **$\color{magenta}{\text{upvote}}$** if you like this kernel!

# **2. Initial exploration of the dataset**

In this section, we are going to set our environment up by importing the necessary libraries, as well as loading our data into a Pandas dataframe.

In [ ]:
# Import the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Read the house data into a data frame
df = pd.read_csv('../input/kc_house_data.csv')

Now, we can go ahead and see how our dataset looks like, while also getting an idea of its statistical properties. 

In [ ]:
# Display the first five observations
df.head()

In [ ]:
# Describe the dataset
df.describe().round(2)

We can see that each entry has an id, i.e. an identification number, as well as a date, i.e. the date on which the transaction took place. These two columns are irrelevant to our analysis; so we will go ahead and remove them.

In [ ]:
# Drop the id and date columns
df = df.drop(['id', 'date'], axis=1)

The attentive reader will already have noticed that there are 21613 observations or entries. Let's confirm it.

In [ ]:
# Display the number of data observations
len(df)

However, we also need to see how many features there are in our dataset.

In [ ]:
# Display the number of starting features
len(df.columns)

Since the number of features is small, it would probably be a good idea to try an alternative implementation, using the normal equation method. Nevertheless, we will go ahead and check the data types and whether they have correctly been interpreted.

In [ ]:
# Check the types of data
df.dtypes

In addition, it is always a good idea to check for any missing values or "NaN" in the dataset. However, our data is very clean.

In [ ]:
# Display the number of null data observations
df.isnull().values.sum()

# **3. Feature selection**

In this section, we are going to use a filter method so as to select which features are going to actually be used by the linear regression algorithm. First of all though, we have to specify which is the target variable we want to predict (price) and which are the features we are going to use (all others).

In [ ]:
# Specify target and features
target = df.iloc[:, 0].name
features = df.iloc[:, 1:].columns.tolist()
features

At this point we are going to check the correlations between our target variable (price) and the features.

In [ ]:
# Correlations of features with target variable
correlations = df.corr()
correlations['price']

Using a filter method, we are going to select which features are to be used subsequently. Although this is arbitrary and contestable, we will remove all features with an absolute correlation with the target variable which is *smaller than 0.2*.

In [ ]:
# Correlations with target variable
cor_target = abs(correlations['price'])

# Display features with correlation < 0.2
removed_features = cor_target[cor_target < 0.2]
removed_features

In [ ]:
# Remove features with correlation < 0.2
df = df.drop(['sqft_lot', 'condition', 'yr_built', 'yr_renovated', 'zipcode', 'long',
              'sqft_lot15'], axis=1)

Furthermore, we also have to ensure that the remaining features are not very highly correlated with each other, i.e. that they are -more or less- independent variables. Let us plot the **Pearson correlation matrix** using Matplotlib and Seaborn.

In [ ]:
# Plot Pearson correlation matrix
fig_1 = plt.figure(figsize=(12, 10))
new_correlations = df.corr()
sns.heatmap(new_correlations, annot=True, cmap='Greens', annot_kws={'size': 8})
plt.title('Pearson Correlation Matrix')
plt.show()

From the above figure, we can see that some features are highly correlated with each other. We will -arbitrarily- search for correlations *above 0.75* and remove the features with the **lowest** correlation with the target variable (price).

In [ ]:
# Determine the highest intercorrelations
highly_correlated_features = new_correlations[new_correlations > 0.75]
highly_correlated_features.fillna('-')

Once more, this is contestable and the result of my own interpretation, but I am of the opinion that although there are sets of variables which are indeed highly correlated, they ultimately represent two independent variables and should be kept as they are. I believe this to be the case with the following sets of variables:
- ("bathrooms", "sqft_living") 
- ("grade", "sqft_living")
- ("grade", sqft_above")

In contrast, I deem that the variable "sqft_living" is not only highly correlated with "sqft_above" (0.877) and "sqft_living15" (0.756), but that it really represents most of the information we need for the linear regression. Indeed, according to the description of the original dataset found on [Kaggle](https://www.kaggle.com/harlfoxem/housesalesprediction), "sqft_above" is the square footage of the house apart from the basement and "sqft_living15" is the square footage of the living room area in 2015 (which implies renovations). It is also useful to note that the variable "grade" is given by the King County grading system and refers to the types of materials used and the quality of workmanship (more info can be found [here](https://info.kingcounty.gov/assessor/esales/Glossary.aspx?type=r#g))

Thus, I chose to remove two more features ("sqft_above" and "sqft_living15") and keep "sqft_living" which has the highest correlation with the target variable (0.702).

In [ ]:
# Remove features which are highly correlated with "sqft_living"
df = df.drop(['sqft_above', 'sqft_living15'], axis=1)

We are ultimately left with **9 features** that will be used for the linear regression. Let us update the features and store their number.

In [ ]:
# Update features and store their length
features = df.iloc[:, 1:].columns.tolist()
len_of_features = len(features)
len_of_features

# **4. Multivariate linear regression**

In this section, we are going to implement our linear regression algorithm. At the very beggining, we are going to **normalize the features** so as to ensure an efficient convergence of the model.

In [ ]:
# Normalize the features
df.iloc[:, 1:] = (df - df.mean())/df.std()

Now we are going to **create the X, y and theta (Θ) matrices**. We could also use for-loops; however, the NumPy library is able to handle numerical computations more efficiently if the operations are vectorized.

In [ ]:
# Create X, y and theta
X = df.iloc[:, 1:]
ones = np.ones([len(df), 1])
X = np.concatenate((ones, X), axis=1)
y = df.iloc[:, 0:1].values
theta = np.zeros([1, len_of_features + 1])

We are also going to store the target variable and check the size of the matrices.

In [ ]:
# Store target
target = y

# Display the size of the matrices
X.shape, y.shape, theta.shape

At this point, let's define the function which computes the **cost function J(Θ)**.

In [ ]:
# Define computecost function
def computecost(X, y, theta):
    H = X @ theta.T
    J = np.power((H - y), 2)
    sum = np.sum(J)/(2 * len(X))
    return sum

It is necessary to set the **parameters** of the model, i.e. the number of the total iterations as well as the learning rate  alpha (α). Feel free to *experiment by changing these parameters* and checking if and how quickly does the model converge.

In [ ]:
# Set iterations and alpha (learning rate)
alpha = 0.01
iterations = 500

We will also define the function which does the **gradient descent**, by minimizing the cost function.

In [ ]:
# Define gradientdescent function
def gradientdescent(X, y, theta, iterations, alpha):
    cost = np.zeros(iterations)
    for i in range(iterations):
        H = X @ theta.T
        theta = theta - (alpha/len(X)) * np.sum(X * (H - y), axis=0)
        cost[i] = computecost(X, y, theta)
    return theta, cost

Now is the most important step in which we will actually do the gradient descent and print the final theta. This is where the "learning" part takes place.

In [ ]:
# Do Gradient Descent and display final theta
final_theta, cost = gradientdescent(X, y, theta, iterations, alpha)
final_theta.round(2)

What is more, we will compute and print the final cost.

In [ ]:
# Compute and display final cost
final_cost = computecost(X, y, final_theta)
final_cost.round(2)

In order to check whether our algorithm converges, we will plot the **Iterations vs. Cost** figure.

In [ ]:
# Plot Iterations vs. Cost figure
fig_2, ax = plt.subplots(figsize=(10, 8))
ax.plot(np.arange(iterations), cost, 'r')
ax.set_xlabel('Iterations')
ax.set_ylabel('Cost')
ax.set_title('Iterations vs. Cost')
plt.show()

Furthermore, we will define a function which computes the **Root Mean Squared Error (RMSE)** so as to measure the differences  between the values predicted by our model and the observed values we had in the beggining of our analysis. RMSE is an absolute measure of fit and it is in the same units as our response variable, i.e. the price in US dollars.

In [ ]:
# Define rmse function
def rmse(target, final_theta):
    predictions = X @ final_theta.T
    return np.sqrt(((predictions[:, 0] - target[:, 0]) ** 2).mean())

# Compute and display Root Mean Squared Error
rmse_val = rmse(target, final_theta)
rmse_val.round(2)

We can see that the RMSE value is high (~216,000 USD). However, if we have in mind that the prices in the original dataset ranged from 75,000 USD to 7,700,000 USD and had a standard deviation of approximately 367,000 USD (see section 2), this RMSE value is not unexpected at all. The best approach would be to implement various machine learning algorithms, compare their results and decide which one works the best for our case.

Finally, let's make a sample prediction in order to check whether our algorithm works as intended. We use the -normalized- features of the first observation in our data which originally had a price of 221,900 USD. 

In [ ]:
# Display sample prediction for first observation
predictions = X @ final_theta.T
str(predictions[0].round(2))

# 5. Conclusion

In this notebook, we applied a multivariate linear regression algorithm so as to predict the house sale prices in King County, USA. The code was greatly detailed and included an exploratory analysis of the dataset and a comprehensive feature selection. In order to get a closer look on what is going on behind the scenes, we used only Pandas and NumPy libraries. 

As I said in the beggining, I am open to any questions, comments and ideas on how to improve the kernel. I hope that it proved useful and that you enjoyed it.

If you want to access the original code and Jupyter Notebook, you can find them on [GitHub](https://github.com/droussis/linear-regression-house-prices).

Apart from Andrew Ng's course (and the contents of Week 1 and 2, in particular), my implementation made use and was influenced by the following websites:
- https://medium.com/we-are-orb/multivariate-linear-regression-in-python-without-scikit-learn-7091b1d45905
- https://towardsdatascience.com/andrew-ngs-machine-learning-course-in-python-linear-regression-dd04fba8e137
- https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b
- https://www.dataquest.io/blog/understanding-regression-error-metrics/
- https://www.statisticshowto.datasciencecentral.com/rmse/
- https://www.theanalysisfactor.com/assessing-the-fit-of-regression-models/